# Setup Inicial

## Bibliotecas utilizadas

In [17]:
import pandas as pd
from pyspark.sql import SparkSession
import json

## Definições gerais

In [18]:
blob_container_name = 'general' # replace with your container name
blob_relative_path_raw = 'raw/mercado_potencial/bacen/' # replace with your relative folder path
blob_relative_path_enriched = 'enriched/mercado_potencial/bacen/' # replace with your relative folder path
linked_service_raw = 'LS_ADLS_RAW_01' # replace with your linked service name
linked_service_enriched = 'LS_ADLS_ENRICHED_01' # replace with your linked service name


ls_raw = mssparkutils.credentials.getPropertiesAll(linked_service_raw)
ls_enriched = mssparkutils.credentials.getPropertiesAll(linked_service_enriched)

converter_dic_raw = json.loads(ls_raw)
converter_dic_enriched = json.loads(ls_enriched)

#coletando o endpoint
end_point_raw = (converter_dic_raw['Endpoint'].split("/"))[2]
end_point_enriched = (converter_dic_enriched['Endpoint'].split("/"))[2]

#Utilizado na leitura via metodo mssparkutils
abfss_path_raw = 'abfss://%s@%s/%s' % (blob_container_name, end_point_raw, blob_relative_path_raw)
abfss_path_enriched = 'abfss://%s@%s/%s' % (blob_container_name, end_point_enriched, blob_relative_path_enriched)

#Utilizados na leitura e gravação dos arquivos via metodo (read_csv ou to_parquet)
#storage_options = {'linked_service':ls_enriched}
#dir_raw  = 'abfss://general/raw/mercado_potencial/bacen/'
#dir_enriched = 'abfss://general/enriched/mercado_potencial/bacen/'


## Funções de utilizadas

In [19]:
def preparaBacen(nm_serie, file):
    """ Prepara dados coletados da API do Banco Central.

        Ajusta o .json captrado direntamente da API do Banco Central trazendo 
        informações de Empregos Formais, Indice ABCR, Selix, Taxa de Câmbio . 
        
    Parâmetros
    ----------
    nm_serie: str 
        string com o nome da serie a ser renomeada.
    file: str
        string com o endereço completo do arquivo no Data Lake
    
    Retorno
    -------
    df : pd.DataFrame
        Dataframe contendo em cada coluna a série de dados contida em cada arquivo (`files`)
        e em formato de serie tempora, com datatime setado como index.
    
    Notes
    -----
    Séries BACEN: https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries

    Para baixar uma série via API, procure o Cedigo da série e substitua em `COD` https://api.bcb.gov.br/dados/serie/bcdata.sgs.COD/dados?formato=FORMATO
    assum como definindo o `FORMATO` como csv ou json

    """
    df = pd.read_json(file , storage_options = {'linked_service' : linked_service_raw})
    df['data'] = pd.to_datetime(df['data'], dayfirst = True)
    df.rename(columns={'valor':nm_serie}, inplace=True)
    df = df.set_index('data', drop=True)
    return(df)

# Execução

## Leitura e ajuste dos Dados da camada raw

In [20]:
#lista todos os arquivos no diretório
files = [i.path for i in mssparkutils.fs.ls(abfss_path_raw)]

# ler e trata dos dados do BACEN
nomes = ['ME_Emprego','IndiceABCR','BC_Selic','BC_Cambio']

pd_mesBACEN = pd.DataFrame()
for i in [0,1,3,2]:#range(len(nomes)):
    pd_mesBACEN = pd.concat([pd_mesBACEN, preparaBacen(nomes[i], files[i])], axis=1)

## Definição do período inicial das séries

In [21]:
# definindo o periodo inicial da série
pd_mesBACEN = pd_mesBACEN['2012':]

## Salvando dados mensais BACEN na camada eriched

In [22]:
# SALVA TABELA EM FORMATO .parquet
pd_mesBACEN.to_parquet(abfss_path_enriched + 'BACEN_mes.parquet', 
                     storage_options = {'linked_service':linked_service_enriched})

# Acesso à documentação

In [23]:
help(preparaBacen)